In [ ]:
# Code to reproduce figure 3 in Forlin, Mebrahtu et al

In [7]:
import warnings
warnings.filterwarnings("ignore")

import ast
import pandas as pd
import numpy as np
import scanpy as sc
import os
import scipy

from tqdm import tqdm
from scipy.stats import mannwhitneyu, ttest_ind
from statsmodels.stats.multitest import multipletests



In [2]:
data_path = #insert data path here 
adata = sc.read(data_path)
sc.pp.normalize_total(adata, target_sum = 1e4)

# Prepare radial plots

In [3]:
df_cytos = pd.read_csv('./data/cytokine_genelist.csv', index_col = [0])

In [4]:
cytokine_genes = df_cytos['genes']
names_cytokine_genes = df_cytos['hallmark']

In [5]:
df_res = pd.DataFrame()
log2fc_df_all = pd.DataFrame()

cell_types = ['B', 'CD4T', 'CD8T', 'Neutrophil', 'Monocyte', 'NK']

for ct in cell_types:
    temp = adata[adata.obs.CellType == ct].copy()
    
    for treat in set(temp.obs.treatment):
        temp2 = temp[temp.obs.treatment == treat]
        df = temp2.to_df()

        if treat != 'Ctrl':
            df_ctrl = temp[temp.obs.treatment == 'Ctrl'].to_df()

        means = []
        medians = []
        results_list = []

        for i, h in enumerate(cytokine_genes):
            h = ast.literal_eval(h)
            summ = np.sum(df[df.columns[df.columns.isin(h)]], axis=1)
            
            if treat != 'Ctrl':
                summ_ctrl = np.sum(df_ctrl[df_ctrl.columns[df_ctrl.columns.isin(h)]], axis=1)
                pvalue = scipy.stats.ttest_ind(summ_ctrl, summ)[1]
                perc_found = len(df_ctrl.columns[df_ctrl.columns.isin(h)]) / len(h)
            else:
                pvalue = 1
                perc_found = 0

            mean = np.mean(summ)
            means.append(mean)
            medians.append(np.median(summ))

            if np.min(means) < 2:
                print(f"Skipping {names_cytokine_genes[i]} for {ct}")
                continue

            results_list.append({
                'mean_value': mean,
                'treatment': treat,
                'hallmark': names_cytokine_genes[i],
                'CellType': ct,
                'pvalue': pvalue,
                'perc_found': perc_found
            })

        df_res = pd.concat([df_res, pd.DataFrame(results_list)])

df_res = df_res.reset_index(drop=True)
df_res['pvalue_adj'] = multipletests(df_res['pvalue'].values, method='fdr_bh')[1]


log2fc_list = []

for ct in cell_types:
    toplot = df_res[df_res.CellType == ct].pivot(index='treatment', columns='hallmark', values='mean_value')

    for treat in toplot.index:
        if treat == 'Ctrl':
            continue

        ctrl = toplot.loc['Ctrl']
        log2fc = np.log2(toplot.loc[treat] / ctrl)
        log2fc = log2fc.replace(-np.inf, 0)  

        log2fc_list.append(pd.DataFrame({'hallmark': log2fc.index, 'log2FC': log2fc.values, 'treatment': treat, 'CellType': ct}))

if log2fc_list:
    log2fc_df_all = pd.concat(log2fc_list, ignore_index=True)

final_df = df_res.merge(log2fc_df_all, on=['hallmark', 'treatment', 'CellType'], how='left')

print("Final merged DataFrame ready!")
final_df = final_df.dropna()
final_df.loc[final_df[final_df.pvalue_adj > 0.05].index, 'log2FC'] = 0

Final merged DataFrame ready!


In [8]:
final_df

,mean_value,treatment,hallmark,CellType,pvalue,perc_found,pvalue_adj,log2FC
0,50.648182,RUX1000,IL1,B,0.249830,1.000000,0.400020,0.000000
1,96.987549,RUX1000,IL4,B,0.004163,1.000000,0.010058,-0.165013
2,47.093254,RUX1000,IL12,B,0.926468,0.936170,1.000000,0.000000
3,205.800369,RUX1000,IL18,B,0.980293,0.981685,1.000000,0.000000
4,31.200260,RUX1000,IL17,B,0.129901,0.968750,0.219584,0.000000
...,...,...,...,...,...,...,...,...
541,69.780273,BMS1000,hallmark_ifnb,NK,0.000379,0.970588,0.001113,-0.529319
542,174.747498,BMS1000,hallmark_tnf,NK,0.060956,0.995000,0.116371,0.000000
543,71.497452,BMS1000,IL8_CXCR1/2,NK,0.400042,0.612903,0.579372,0.000000
544,42.633381,BMS1000,HLAs-Type I,NK,0.392049,1.000000,0.572890,0.000000


In [11]:
#final_df.to_csv('./data/Figure3_radialPlots_toR.csv')